# Anime Dataset 2023 - Advanced Exploratory Data Analysis

This notebook performs a thorough and academically oriented EDA for the Anime Dataset 2023.

Role and goal:
- Role: Producer and data analyst
- Goal: Understand which factors are associated with anime success and prepare features for a later Success Score model

Target metric (conceptual model):
- Score = a*Type + b*Episodes + c*Genre_Action + ... + constant + error

The notebook is structured with clear sections, explicit assumptions and systematic diagnostics.


## 1. Data understanding and feature dictionary

This section documents the meaning of each column in the dataset and groups features into logical categories.
The content below is adapted from the original raw analysis file.


## Definition of Each Features

### 1. Basic Information and Identification (Basic Identification & Description)
- **anime_id**: Unique ID for each anime.
- **Name**: The anime's title in its original language.
- **English name**: The official English title of the anime.
- **Other name**: Local title, native title, or alternative titles of the anime.
- **Synopsis**: Brief description or plot summary of the anime.
- **Genres**: Genres of the anime, separated by commas.
- **Image URL**: URL of the anime's poster or thumbnail image.

### 2. Production and Technical Details (Production & Technical Details)
- **Type**: Type of anime (e.g., TV, Movie, OVA, etc.).
- **Source**: Original source material of the anime (e.g., Manga, Light Novel, Original, etc.).
- **Producers**: Companies or individuals that produced or funded the anime.
- **Studios**: Animation studios that produced the anime.
- **Licensors**: Companies that licensed the anime for distribution (especially outside Japan).
- **Episodes**: Total number of episodes in the anime.
- **Duration**: Duration of each episode.

### 3. Broadcast and Release Information (Release & Airing Information)
- **Aired**: Airing period or release date of the anime.
- **Premiered**: The season and year the anime first premiered (e.g., Fall 2023).
- **Status**: Current status of the anime (e.g., Finished Airing, Currently Airing, Not Yet Aired).

### 4. Viewer Engagement and Performance Metrics (Audience Engagement & Performance Metrics)
- **Score**: Average score given to the anime by users.
- **Rating**: Age rating of the anime (e.g., PG-13, R, G).
- **Rank**: Overall ranking of the anime based on score or other criteria.
- **Popularity**: Popularity ranking of the anime (based on member count or views).
- **Favorites**: Number of users who have marked the anime as a favorite.
- **Scored By**: Number of users who have rated/scored the anime.
- **Members**: Total number of users who have added the anime to their list on the platform.

## 2. Setup and configuration


In [2]:
import os
import re
from itertools import chain

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 180)

PLOT_DIR = "plots"
os.makedirs(PLOT_DIR, exist_ok=True)


# Plotly global aesthetic setup:
import copy
import plotly.io as pio
# Try to create a safe custom template based on plotly_white; fall back if unavailable
try:
    base_template = copy.deepcopy(pio.templates["plotly_white"])
    # remove gridlines on both axes if layout exists
    if hasattr(base_template, 'layout'):
        try:
            base_template.layout.xaxis.update(dict(showgrid=False, zeroline=False))
        except Exception:
            pass
        try:
            base_template.layout.yaxis.update(dict(showgrid=False, zeroline=False))
        except Exception:
            pass
        base_template.layout.font = dict(family="Arial", size=12, color="#222222")
        pio.templates["custom_nogrid"] = base_template
        px.defaults.template = "custom_nogrid"
    else:
        px.defaults.template = "plotly_white"
except Exception:
    # fallback to default white template if creating custom fails
    px.defaults.template = "plotly_white"
# Set a pleasant discrete color sequence and continuous scale
px.defaults.color_discrete_sequence = px.colors.qualitative.Plotly
px.defaults.color_continuous_scale = px.colors.sequential.Viridis


## 3. EDA methodology

This EDA follows a structured workflow:

1. Structural overview
   - Inspect shape, schema and basic distributions.
2. Data quality assessment
   - Missingness, NA like tokens, suspicious zeros, type conversions and outliers.
3. Univariate analysis
   - Distributions of key numeric and categorical features.
4. Bivariate analysis with respect to the target
   - Score vs numeric features
   - Score vs categorical features
5. Multivariate structure
   - Correlation matrix and simple multicollinearity diagnostics.
6. Segment based and diagnostic analysis
   - Year, Type and Studio segments.
7. Feature readiness
   - Document which features can be safely used for modeling.


## 4. Load dataset and structural overview

In [3]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path("..") / "data" / "raw" / "anime-dataset-2023.csv"

df = pd.read_csv(DATA_PATH)


print("Shape (rows, columns):", df.shape)
print("\nColumns:")
print(df.columns.tolist())


Shape (rows, columns): (24905, 24)

Columns:
['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres', 'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status', 'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating', 'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL']


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   anime_id      24905 non-null  int64 
 1   Name          24905 non-null  object
 2   English name  24905 non-null  object
 3   Other name    24905 non-null  object
 4   Score         24905 non-null  object
 5   Genres        24905 non-null  object
 6   Synopsis      24905 non-null  object
 7   Type          24905 non-null  object
 8   Episodes      24905 non-null  object
 9   Aired         24905 non-null  object
 10  Premiered     24905 non-null  object
 11  Status        24905 non-null  object
 12  Producers     24905 non-null  object
 13  Licensors     24905 non-null  object
 14  Studios       24905 non-null  object
 15  Source        24905 non-null  object
 16  Duration      24905 non-null  object
 17  Rating        24905 non-null  object
 18  Rank          24905 non-null  object
 19  Popu

In [5]:
print("First 10 rows:")
display(df.head(10))


First 10 rows:


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Producers,Licensors,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",spring 1998,Finished Airing,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",UNKNOWN,Finished Airing,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",spring 1998,Finished Airing,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",summer 2002,Finished Airing,"Bandai Visual, Dentsu, Victor Entertainment, T...","Funimation, Bandai Entertainment",Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",fall 2004,Finished Airing,"TV Tokyo, Dentsu",Illumitoon Entertainment,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...
5,15,Eyeshield 21,UNKNOWN,アイシールド21,7.92,Sports,"Shy, reserved, and small-statured, Deimon High...",TV,145.0,"Apr 6, 2005 to Mar 19, 2008",spring 2005,Finished Airing,"TV Tokyo, Nihon Ad Systems, TV Tokyo Music, Sh...","VIZ Media, Sentai Filmworks",Gallop,Manga,23 min per ep,PG-13 - Teens 13 or older,688.0,1252,1997,86524.0,177688,https://cdn.myanimelist.net/images/anime/1079/...
6,16,Hachimitsu to Clover,Honey and Clover,ハチミツとクローバー,8.0,"Comedy, Drama, Romance","Yuuta Takemoto, a sophomore at an arts college...",TV,24.0,"Apr 15, 2005 to Sep 27, 2005",spring 2005,Finished Airing,"Dentsu, Genco, Fuji TV, Asmik Ace, Shueisha","VIZ Media, Discotek Media",J.C.Staff,Manga,23 min per ep,PG-13 - Teens 13 or older,589.0,862,4136,81747.0,260166,https://cdn.myanimelist.net/images/anime/1301/...
7,17,Hungry Heart: Wild Striker,UNKNOWN,ハングリーハート Wild Striker,7.55,"Comedy, Slice of Life, Sports",As the younger brother of Japanese soccer star...,TV,52.0,"Sep 11, 2002 to Sep 10, 2003",fall 2002,Finished Airing,UNKNOWN,UNKNOWN,Nippon Animation,Manga,23 min per ep,PG-13 - Teens 13 or older,1551.0,4212,237,12960.0,24172,https://cdn.myanimelist.net/images/anime/12/49...
8,18,Initial D Fourth Stage,UNKNOWN,頭文字〈イニシャル〉D FOURTH STAGE,8.16,"Action, Drama",Takumi Fujiwara finally joins Ryousuke and Kei...,TV,24.0,"Apr 17, 2004 to Feb 18, 2006",spring 2004,Finished Airing,"OB Planning, Studio Jack",Funimation,A.C.G.T.,Manga,27 min per ep,PG-13 - Teens 13 or older,393.0,1273,1237,97878.0,173710,https://cdn.myanimelist.net/images/anime/9/105...
9,19,Monster,Monster,モンスター,8.87,"Drama, Mystery, Suspense","Dr. Kenzou Tenma, an elite neurosurgeon recent...",TV,74.0,"Apr 7, 2004 to Sep 28, 2005",spring 2004,Finished Airing,"VAP, Shogakukan-Shueisha Productions, Nippon T...",VIZ Media,Madhouse,Manga,24 min per ep,R+ - M

In [6]:
print("Numeric columns summary:")
display(df.describe(include=[np.number]).T)

print("\nCategorical columns summary:")
display(df.describe(include=["object"]).T)

Numeric columns summary:


,count,mean,std,min,25%,50%,75%,max
anime_id,24905.0,29776.709014,17976.076290,1.0,10507.0,34628.0,45240.0,55735.0
Popularity,24905.0,12265.388356,7187.428393,0.0,6040.0,12265.0,18491.0,24723.0
Favorites,24905.0,432.595222,4353.181647,0.0,0.0,1.0,18.0,217606.0
Members,24905.0,37104.960008,156825.237061,0.0,209.0,1056.0,9326.0,3744541.0



Categorical columns summary:


,count,unique,top,freq
Name,24905,24901,Azur Lane,2
English name,24905,10134,UNKNOWN,14577
Other name,24905,23796,UNKNOWN,128
Score,24905,567,UNKNOWN,9213
Genres,24905,1006,UNKNOWN,4929
Synopsis,24905,20113,No description available for this anime.,4535
Type,24905,7,TV,7597
Episodes,24905,252,1.0,11532
Aired,24905,15213,Not available,915
Premiered,24905,244,UNKNOWN,19399


## 5. Data quality assessment

This section focuses on data quality:
- Missing values and NA like tokens
- Duplicates
- Type conversions
- Suspicious zeros
- Outlier inspection


### 5.1 Missing values by column

In [7]:
missing_ratio = df.isna().mean().sort_values(ascending=False)
print("Missing value ratio by column:")
display(missing_ratio)


Missing value ratio by column:


anime_id        0.0
Name            0.0
Members         0.0
Scored By       0.0
Favorites       0.0
Popularity      0.0
Rank            0.0
Rating          0.0
Duration        0.0
Source          0.0
Studios         0.0
Licensors       0.0
Producers       0.0
Status          0.0
Premiered       0.0
Aired           0.0
Episodes        0.0
Type            0.0
Synopsis        0.0
Genres          0.0
Score           0.0
Other name      0.0
English name    0.0
Image URL       0.0
dtype: float64

### 5.2 Duplicates by anime_id

In [8]:
dup_count = df.duplicated(subset=["anime_id"]).sum()
print("Number of duplicated anime_id:", dup_count)

if dup_count > 0:
    print("\nExamples of duplicated anime_id:")
    display(df[df.duplicated(subset=["anime_id"], keep=False)].sort_values("anime_id").head(10))


Number of duplicated anime_id: 0


### 5.3 NA like tokens in object columns

Some string tokens effectively represent missing values but are not coded as NaN.
We detect and optionally normalize these tokens to proper NaN.


#### 5.3.1 Systematic scan for NaN-like tokens

To make the treatment of NaN-like tokens more systematic and auditable,
we build a compact report that scans the top values of every object column
and aggregates potential NaN-like tokens with their frequencies.


In [9]:
def scan_na_like_tokens(df: pd.DataFrame, top_k: int = 50) -> pd.DataFrame:
    """Build a long-form report of tokens observed in object columns.

    For each object column, we take the top_k most frequent values (including
    non-NaN) and record:

    - column: column name
    - token: normalized string representation of the value
    - count: absolute frequency
    - ratio: percentage of rows in that column

    This does not decide which tokens are 'NA-like' by itself, but it is the
    basis for applying rule-based or business-informed filters.
    """
    obj_cols = df.select_dtypes(include=["object"]).columns
    rows = []

    for col in obj_cols:
        vc = df[col].value_counts(dropna=False)
        top_vals = vc.head(top_k)
        total = len(df[col])

        for val, cnt in top_vals.items():
            if pd.isna(val):
                token_str = "<NaN>"
            else:
                token_str = str(val).strip()
            ratio = cnt / total if total else 0.0
            rows.append({
                "column": col,
                "token": token_str,
                "count": int(cnt),
                "ratio": round(ratio * 100, 2),
            })

    report = pd.DataFrame(rows)
    return report

na_scan = scan_na_like_tokens(df, top_k=20)


## Alternative views for `na_scan` (readable / paginated / export)

When `na_scan` is long, the plain DataFrame output can be hard to read. Below are four options you can run:

1. Scrollable HTML table (compact).
2. Pandas `Styler` with a fixed-height block and sticky header.
3. Simple pagination helper to view pages of rows (adjust page size).
4. Export to CSV for external viewing (Excel/VS Code).

Run the code cell after this to try the options.

In [10]:
import numpy as np
import pandas as pd
from IPython.display import HTML, display

###########################################################################
# Scrollable HTML block (compact, preserves styles)
###########################################################################
def show_scrollable_html(df, max_height=400, width=600):
    """
    Render a DataFrame inside a scrollable HTML block.
    Automatically hides column `is_na_like_rule` if present.
    """
    # Tự động bỏ cột is_na_like_rule nếu tồn tại
    if "is_na_like_rule" in df.columns:
        df = df.drop(columns=["is_na_like_rule"])

    html = df.to_html(index=False, classes="table table-striped table-sm")
    style = (
        f"max-height:{max_height}px; "
        f"overflow:auto; border:1px solid #ddd; padding:6px; "
        f"width:{width}px;"
    )
    wrapper = f"<div style='{style}'>{html}</div>"
    display(HTML(wrapper))

# Ví dụ dùng để hiển thị na_scan:
show_scrollable_html(na_scan)


column,token,count,ratio
Name,Azur Lane,2,0.01
Name,Utopia,2,0.01
Name,Souseiki,2,0.01
Name,Awakening,2,0.01
Name,Cowboy Bebop,1,0.00
Name,Noctambulist,1,0.00
Name,Mizushiri Yoriko no Yorinuki Yoriko-san,1,0.00
Name,Umi,1,0.00
Name,Big Hits! ID (Yoriko Mizushiri),1,0.00
Name,A Sea Doesn't Tell Much,1,0.00


In [11]:
def is_potential_na_token(t: str) -> bool:
    t_norm = t.strip().lower()

    # known canonical NA-like tokens
    if t_norm in {
        "", "na", "n/a", "none", "null", "nan",
        "-", "?", "unknown", "not available", "n.a.", "n a",
        "not available",
        "no description available for this anime.",
        "https://cdn.myanimelist.net/img/sp/icon/apple-touch-icon-256.png"
    }:
        return True

    # short tokens with mostly punctuation can also be suspicious
    if len(t_norm) <= 2 and any(ch in t_norm for ch in ["?", "-", "."]):
        return True
    return False


na_scan["is_na_like_rule"] = na_scan["token"].apply(
    lambda x: False if x == "<NaN>" else is_potential_na_token(x)
)

na_candidates_rule = na_scan[na_scan["is_na_like_rule"]].copy()
na_candidates_rule = na_candidates_rule.sort_values(
    ["token", "column", "count"],
    ascending=[True, True, False]
)
na_candidates_rule = na_candidates_rule.reset_index(drop=True)


print("Rule-flagged potential NA-like tokens across object columns:")
display(
    na_candidates_rule
        .drop(columns=["is_na_like_rule"], errors="ignore")
        .head(100)
)


Rule-flagged potential NA-like tokens across object columns:


,column,token,count,ratio
0,Synopsis,No description available for this anime.,4535,18.21
1,Aired,Not available,915,3.67
2,English name,UNKNOWN,14577,58.53
3,Episodes,UNKNOWN,611,2.45
4,Genres,UNKNOWN,4929,19.79
5,Licensors,UNKNOWN,20170,80.99
6,Other name,UNKNOWN,128,0.51
7,Premiered,UNKNOWN,19399,77.89
8,Producers,UNKNOWN,13350,53.60
9,Rank,UNKNOWN,4612,18.52


We can then flag potential NaN-like tokens by applying a simple rule-based detector.
This makes the identification of NA-like patterns reproducible and inspectable.


In [12]:
# 2. Tạo tập token sẽ convert thành NaN (dựa trên rule vừa flag)
NA_TOKENS_RULE = set(
    t for t in na_candidates_rule["token"].unique()
    if t != "<NaN>"
)

print("Tokens to be treated as NA (from rule):")
print(sorted(NA_TOKENS_RULE))

# 3. Tạo dataframe mới, không đụng df gốc
df_na_normalized = df.copy()

# Lấy lại danh sách cột object trên df mới
obj_cols = df_na_normalized.select_dtypes(include=["object"]).columns

# 4. Normalize NA-like tokens sang np.nan trên bản copy
for col in obj_cols:
    df_na_normalized[col] = df_na_normalized[col].replace(list(NA_TOKENS_RULE), np.nan)

# 5. (Optional) Lưu ra CSV sạch riêng
# from pathlib import Path
# DATA_OUT = Path("..") / "data" / "processed" / "anime_dataset_2023_na_normalized.csv"
# df_na_normalized.to_csv(DATA_OUT, index=False)

Tokens to be treated as NA (from rule):
['No description available for this anime.', 'Not available', 'UNKNOWN', 'Unknown', 'https://cdn.myanimelist.net/img/sp/icon/apple-touch-icon-256.png']


### 5.4 Type conversion


In [13]:
import pandas as pd

# 0. Choose base for type conversion (do not touch df directly)
base_df = df_na_normalized.copy() if "df_na_normalized" in globals() else df.copy()

# 1. Create df_typed for typed numeric conversion
df_typed = base_df.copy()

# 2. Define numeric columns
numeric_cols_to_convert = [
    "Score",
    "Episodes",
    "Rank",
    "Scored By",
    "Popularity",
    "Favorites",
    "Members",
    "anime_id",
]

# 3. Convert selected columns to numeric in-place
for col in numeric_cols_to_convert:
    if col in df_typed.columns:
        df_typed[col] = pd.to_numeric(df_typed[col], errors="coerce")

# 4. Diagnostic: detect values that failed conversion (OPTIONAL)
for col in ["Score", "Episodes", "Rank", "Scored By"]:
    if col in df_typed.columns:
        # Values that are now NaN but were not empty strings before
        mask_invalid = df_typed[col].isna() & base_df[col].notna()
        invalid_raw_values = base_df.loc[mask_invalid, col]
        print(f"\nInvalid values in '{col}' after conversion: {invalid_raw_values.nunique()} unique values")
        display(
            invalid_raw_values.value_counts().head(20).to_frame("count")
        )



Invalid values in 'Score' after conversion: 0 unique values


,count
Score,



Invalid values in 'Episodes' after conversion: 0 unique values


,count
Episodes,



Invalid values in 'Rank' after conversion: 0 unique values


,count
Rank,



Invalid values in 'Scored By' after conversion: 0 unique values


,count
Scored By,


In [14]:
print("Numeric columns summary:")
display(df_typed.describe(include=[np.number]).T)

Numeric columns summary:


,count,mean,std,min,25%,50%,75%,max
anime_id,24905.0,29776.709014,17976.076290,1.00,10507.00,34628.00,45240.00,55735.0
Score,15692.0,6.380890,0.928696,1.85,5.73,6.39,7.06,9.1
Episodes,24294.0,14.913518,48.581251,1.00,1.00,2.00,13.00,3057.0
Rank,20293.0,9960.770167,5856.990951,0.00,4885.00,9958.00,15032.00,20104.0
Popularity,24905.0,12265.388356,7187.428393,0.00,6040.00,12265.00,18491.00,24723.0
Favorites,24905.0,432.595222,4353.181647,0.00,0.00,1.00,18.00,217606.0
Scored By,15692.0,29974.376880,116934.792749,100.00,384.00,1760.00,10855.50,2660903.0
Members,24905.0,37104.960008,156825.237061,0.00,209.00,1056.00,9326.00,3744541.0


### 5.5 Suspicious zeros in numeric columns

Some numeric columns may use 0 as a placeholder for missing or not applicable values.
We generate a zero report to identify candidates for zero to NaN recoding.


In [15]:
base_df = df_typed.copy()


num_cols = base_df.select_dtypes(include=[np.number]).columns

rows = []
n_total = len(base_df)

for c in num_cols:
    series = base_df[c]
    zero_count = int((series == 0).sum())
    nan_count = int(series.isna().sum())
    rows.append({
        "column": c,
        "zero_count": zero_count,
        "zero_pct": round(zero_count / n_total * 100, 2) if n_total else 0.0,
        "nan_count": nan_count,
    })

zero_report = pd.DataFrame(rows).sort_values(
    ["zero_pct", "zero_count"],
    ascending=False
)

print("Zero report for numeric columns:")
display(zero_report.head(30))


Zero report for numeric columns:


,column,zero_count,zero_pct,nan_count
5,Favorites,10808,43.40,0
3,Rank,187,0.75,4612
4,Popularity,187,0.75,0
7,Members,186,0.75,0
0,anime_id,0,0.00,0
1,Score,0,0.00,9213
2,Episodes,0,0.00,611
6,Scored By,0,0.00,9213


In [16]:
import numpy as np

# 1. Specify which numeric columns should treat zero as missing
cols_treat_zero_as_nan = ["Rank", "Popularity"]  # example

# 2. Sanity check: show zero and NaN counts BEFORE replacement
print("Zero and NaN counts BEFORE treating zero as missing:")
stats_before = []
for col in cols_treat_zero_as_nan:
    if col in df_typed.columns:
        s = df_typed[col]
        stats_before.append({
            "column": col,
            "zero_count": int((s == 0).sum()),
            "nan_count": int(s.isna().sum()),
            "non_null_count": int(s.notna().sum()),
        })

stats_before_df = pd.DataFrame(stats_before)
display(stats_before_df)

# 3. Replace zero with NaN for selected columns on df_typed
for col in cols_treat_zero_as_nan:
    if col in df_typed.columns:
        df_typed[col] = df_typed[col].replace(0, np.nan)

# 4. Check again: zero and NaN counts AFTER replacement
print("Zero and NaN counts AFTER treating zero as missing:")
stats_after = []
for col in cols_treat_zero_as_nan:
    if col in df_typed.columns:
        s = df_typed[col]
        stats_after.append({
            "column": col,
            "zero_count": int((s == 0).sum()),
            "nan_count": int(s.isna().sum()),
            "non_null_count": int(s.notna().sum()),
        })

stats_after_df = pd.DataFrame(stats_after)
display(stats_after_df)


Zero and NaN counts BEFORE treating zero as missing:


,column,zero_count,nan_count,non_null_count
0,Rank,187,4612,20293
1,Popularity,187,0,24905


Zero and NaN counts AFTER treating zero as missing:


,column,zero_count,nan_count,non_null_count
0,Rank,0,4799,20106
1,Popularity,0,187,24718


# Visulize after Normalization

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import missingno as msno
from IPython.display import display


def missing_data_dashboard(
    df_raw: pd.DataFrame,
    df_clean: pd.DataFrame | None = None,
    max_rows_matrix: int = 2000,
) -> None:
    """
    Full missing data analysis.

    Parameters
    ----------
    df_raw : pd.DataFrame
        Original dataframe (before NA-like normalization).
    df_clean : pd.DataFrame or None
        Dataframe after NA-like normalization. If None, df_raw is used.
    max_rows_matrix : int
        Maximum number of rows sampled for the missingness matrix heatmap.
    """

    # 1. Choose base_df for missingness analysis
    base_df = df_clean.copy() if df_clean is not None else df_raw.copy()

    print("Data shape for missing analysis:", base_df.shape)

    # 2. Bar chart - missing ratio by column
    missing_ratio = base_df.isna().mean().sort_values(ascending=False)

    fig = px.bar(
        x=missing_ratio.index,
        y=missing_ratio.values,
        labels={"x": "Column", "y": "Missing ratio"},
        title="Missing data ratio by column",
    )
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

    # 3. Correlation heatmap of missingness
    cols_focus = [
        "Score", "Episodes", "Rank", "Producers", "Studios", 
        "Premiered", "Duration"
    ]

    cols_focus = [c for c in cols_focus if c in base_df.columns]

    missing_corr = base_df[cols_focus].isna().corr()

    fig = px.imshow(
        missing_corr,
        text_auto=".2f",
        aspect="auto",
        labels={"x": "Column", "y": "Column", "color": "Correlation"},
        title="Correlation of missingness (Selected important columns)",
    )
    fig.show()

    # 4. Top columns by missing ratio
    top_missing = (
        missing_ratio.reset_index()
        .rename(columns={"index": "column", 0: "missing_ratio"})
    )
    top_missing["missing_ratio"] = (top_missing["missing_ratio"] * 100).round(2)
    print("Top columns by missing ratio (%):")
    display(top_missing.head(20))


In [18]:
missing_data_dashboard(df_raw=df, df_clean=df_typed)

Data shape for missing analysis: (24905, 24)


Top columns by missing ratio (%):


,column,missing_ratio
0,Licensors,80.99
1,Premiered,77.89
2,English name,58.53
3,Producers,53.60
4,Studios,42.26
5,Scored By,36.99
6,Score,36.99
7,Genres,19.79
8,Rank,19.27
9,Synopsis,18.21


### 5.6 Outlier inspection for Score and Episodes

In [19]:
print("Score summary:")
display(df_typed["Score"].describe())

print("\nEpisodes summary:")
display(df_typed["Episodes"].describe())


Score summary:


count    15692.000000
mean         6.380890
std          0.928696
min          1.850000
25%          5.730000
50%          6.390000
75%          7.060000
max          9.100000
Name: Score, dtype: float64


Episodes summary:


count    24294.000000
mean        14.913518
std         48.581251
min          1.000000
25%          1.000000
50%          2.000000
75%         13.000000
max       3057.000000
Name: Episodes, dtype: float64

In [20]:
import pandas as pd
import plotly.express as px
from pathlib import Path

PLOT_DIR = Path("plots")
PLOT_DIR.mkdir(parents=True, exist_ok=True)

def plot_univariate_distribution_plotly(
    df: pd.DataFrame,
    col: str,
    log_x: bool = False,
    nbins: int = 40,
    save: bool = False,
    suffix: str = ""
) -> None:
    """
    Plot histogram and boxplot for a numeric column using Plotly.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe.
    col : str
        Numeric column name to visualize.
    log_x : bool, default False
        Whether to use log scale on the x-axis.
    nbins : int, default 40
        Number of histogram bins.
    save : bool, default False
        If True, save figures into PLOT_DIR as HTML files.
    suffix : str, default ""
        Optional suffix for saved file names.
    """
    if col not in df.columns:
        raise ValueError(f"Column '{col}' not found in dataframe.")

    # Work on a clean 1D series
    series = df[col].dropna()

    # For log-x, remove non-positive values
    if log_x:
        series = series[series > 0]

    if series.empty:
        print(f"No valid values to plot for column '{col}'.")
        return

    # Histogram - use the cleaned series instead of the full df
    fig_hist = px.histogram(
        x=series,
        nbins=nbins,
        title=f"Histogram of {col}",
        labels={"x": col, "y": "Count"},
    )
    if log_x:
        fig_hist.update_xaxes(type="log")
    fig_hist.show()

    # Boxplot - also on the cleaned series
    fig_box = px.box(
        x=series,
        title=f"Boxplot of {col}",
        labels={"x": col},
    )
    if log_x:
        fig_box.update_xaxes(type="log")
    fig_box.show()

    # Optional saving
    if save:
        safe_suffix = f"_{suffix}" if suffix else ""
        out_hist = PLOT_DIR / f"{col.lower()}_hist{safe_suffix}.html"
        out_box = PLOT_DIR / f"{col.lower()}_box{safe_suffix}.html"
        fig_hist.write_html(out_hist)
        fig_box.write_html(out_box)
        print(f"Saved Plotly figures for {col} to:")
        print(f" - {out_hist}")
        print(f" - {out_box}")


In [21]:
plot_univariate_distribution_plotly(df_typed, "Score", log_x=False, nbins=40, save=False)
plot_univariate_distribution_plotly(df_typed, "Episodes", log_x=True, nbins=40, save=False)


### 5.6.2. Extended outlier inspection

In this subsection we perform targeted outlier diagnostics for additional numeric features that are either critical for modeling or prone to data issues.

**Columns inspected**

- Score
- Episodes
- Duration_minutes
- Members
- Scored By
- Favorites
- aired_year
- Rank
- Popularity

**Outlier categories**

- Suspected error  
  - Values that violate basic domain constraints  
  - Examples  
    - Score < 1 or Score > 10  
    - Episodes <= 0 or Episodes > 1000  
    - Duration_minutes <= 0 or Duration_minutes > 400  
    - aired_year < 1960 or aired_year > 2025  
    - Any negative counts in Members, Scored By, Favorites  

- Legit extreme  
  - Values that are rare but still lie within the valid domain  
  - Operational definition in this notebook  
    - Within the valid domain  
    - Outside the IQR envelope [Q1 - 1.5 * IQR, Q3 + 1.5 * IQR]  

The goal of this diagnostic is not to automatically remove all extreme values, but to separate potential data errors from rare but legitimate extreme observations and to document these decisions for the later modeling phase.


In [22]:
# Configuration for extended outlier inspection
# Sửa tên cột cho khớp với dataframe của bạn nếu cần
extended_outlier_config = {
    "Score": {
        "valid_min": 1.0,
        "valid_max": 10.0,
        "description": "Main target on a 1 to 10 scale"
    },
    "Episodes": {
        "valid_min": 1,
        # Giới hạn trên mang tính nghiệp vụ, có thể chỉnh lại
        "valid_max": 1000,
        "description": "Number of episodes, should be positive and reasonably bounded"
    },
    "Duration_minutes": {
        # Nếu bạn chưa có cột này thì có thể tạo trước đó,
        # hoặc đổi tên sang cột duration tương ứng trong df
        "valid_min": 1,
        "valid_max": 400,
        "description": "Duration per episode in minutes"
    },
    "Members": {
        "valid_min": 0,
        "valid_max": None,  # Unbounded on the upper side, we only flag negatives as errors
        "description": "Number of users who added the anime to their list"
    },
    "Scored By": {
        "valid_min": 0,
        "valid_max": None,
        "description": "Number of users who rated the anime"
    },
    "Favorites": {
        "valid_min": 0,
        "valid_max": None,
        "description": "Number of users who marked the anime as favorite"
    },
    "aired_year": {
        # Cột này thường được tạo từ Aired/Premiered trước đó
        "valid_min": 1960,
        "valid_max": 2025,
        "description": "Release year extracted from Aired or Premiered"
    },
    "Rank": {
        "valid_min": 1,
        "valid_max": None,
        "description": "Overall ranking, should be positive if present"
    },
    "Popularity": {
        "valid_min": 1,
        "valid_max": None,
        "description": "Popularity ranking, should be positive if present"
    },
}


def extended_outlier_report(
    df: pd.DataFrame,
    config: dict = extended_outlier_config,
    iqr_k: float = 1.5,
    top_n: int = 5,
    verbose: bool = True,
):
    """
    Run extended outlier diagnostics for selected numeric columns.

    For each configured column:
    - Coerce to numeric (errors as NaN)
    - Compute basic stats and IQR envelope
    - Flag:
        - suspected_error: violates valid_min or valid_max (if defined)
        - legit_extreme: within valid range but outside IQR envelope
    - Print a compact report and return a summary dict
    """
    summary = {}

    id_candidates = [c for c in ["anime_id", "Name", "name"] if c in df.columns]

    for col, rules in config.items():
        if col not in df.columns:
            if verbose:
                print(f"[skip] Column '{col}' not found in dataframe")
            continue

        if verbose:
            print(f"\n=== Extended outlier inspection for '{col}' ===")
            if "description" in rules:
                print(f"Description: {rules['description']}")

        s_raw = df[col]
        s = pd.to_numeric(s_raw, errors="coerce")

        valid_min = rules.get("valid_min", None)
        valid_max = rules.get("valid_max", None)

        n_total = s.shape[0]
        n_non_null = s.notna().sum()

        if n_non_null == 0:
            if verbose:
                print(f"No non null values in column '{col}'. Skipping.")
            continue

        # Basic stats
        q1 = s.quantile(0.25)
        q3 = s.quantile(0.75)
        iqr = q3 - q1
        iqr_low = q1 - iqr_k * iqr
        iqr_high = q3 + iqr_k * iqr

        col_min = s.min()
        col_max = s.max()

        # Define masks
        suspected_error_mask = pd.Series(False, index=s.index)
        if valid_min is not None:
            suspected_error_mask |= s < valid_min
        if valid_max is not None:
            suspected_error_mask |= s > valid_max

        # Legit extreme: within valid domain (or domain not specified),
        # but outside IQR envelope
        in_domain_mask = pd.Series(True, index=s.index)
        if valid_min is not None:
            in_domain_mask &= s >= valid_min
        if valid_max is not None:
            in_domain_mask &= s <= valid_max

        legit_extreme_mask = in_domain_mask & (
            (s < iqr_low) | (s > iqr_high)
        )

        # Remove suspected errors from legit extremes
        legit_extreme_mask &= ~suspected_error_mask

        n_suspected = suspected_error_mask.sum()
        n_legit_extreme = legit_extreme_mask.sum()

        if verbose:
            print(f"Non null count      : {n_non_null} / {n_total}")
            print(f"Min / Max           : {col_min} / {col_max}")
            print(f"Q1 / Q3             : {q1} / {q3}")
            print(f"IQR                 : {iqr}")
            print(f"IQR envelope        : [{iqr_low}, {iqr_high}]")
            print(f"Suspected errors    : {n_suspected}")
            print(f"Legit extremes      : {n_legit_extreme}")

        # Prepare samples for inspection
        cols_to_show = id_candidates + [col] if id_candidates else [col]

        suspected_examples = df.loc[suspected_error_mask, cols_to_show].head(top_n)
        legit_extreme_examples = df.loc[legit_extreme_mask, cols_to_show].head(top_n)

        if verbose and n_suspected > 0:
            print("\nSample suspected errors:")
            display(suspected_examples)

        if verbose and n_legit_extreme > 0:
            print("\nSample legit extremes:")
            display(legit_extreme_examples)

        summary[col] = {
            "n_total": int(n_total),
            "n_non_null": int(n_non_null),
            "min": float(col_min),
            "max": float(col_max),
            "q1": float(q1),
            "q3": float(q3),
            "iqr": float(iqr),
            "iqr_low": float(iqr_low),
            "iqr_high": float(iqr_high),
            "n_suspected_errors": int(n_suspected),
            "n_legit_extremes": int(n_legit_extreme),
            "suspected_error_mask": suspected_error_mask,
            "legit_extreme_mask": legit_extreme_mask,
        }

    return summary


# Gọi hàm để chạy extended outlier inspection
extended_outlier_summary = extended_outlier_report(df)



=== Extended outlier inspection for 'Score' ===
Description: Main target on a 1 to 10 scale
Non null count      : 15692 / 24905
Min / Max           : 1.85 / 9.1
Q1 / Q3             : 5.73 / 7.06
IQR                 : 1.3299999999999992
IQR envelope        : [3.7350000000000017, 9.054999999999998]
Suspected errors    : 0
Legit extremes      : 68

Sample legit extremes:


,anime_id,Name,Score
387,413,Hametsu no Mars,2.22
1215,1345,Soujuu Senshi Psychic Wars,3.08
1441,1585,Spectral Force,3.55
1649,1812,Hanoka,3.33
2545,2776,Generation of Chaos,3.07



=== Extended outlier inspection for 'Episodes' ===
Description: Number of episodes, should be positive and reasonably bounded
Non null count      : 24294 / 24905
Min / Max           : 1.0 / 3057.0
Q1 / Q3             : 1.0 / 13.0
IQR                 : 12.0
IQR envelope        : [-17.0, 31.0]
Suspected errors    : 10
Legit extremes      : 2846

Sample suspected errors:


,anime_id,Name,Episodes
2266,2471,Doraemon (1979),1787.0
4525,6277,Manga Nippon Mukashibanashi (1976),1471.0
5267,8213,Hoka Hoka Kazoku,1428.0
5974,9947,Lan Mao,3057.0
6107,10241,Sekai Monoshiri Ryoko,1006.0



Sample legit extremes:


...

,anime_id,Name,Favorites
0,1,Cowboy Bebop,78525
1,5,Cowboy Bebop: Tengoku no Tobira,1448
2,6,Trigun,15035
3,7,Witch Hunter Robin,613
5,15,Eyeshield 21,1997


[skip] Column 'aired_year' not found in dataframe

=== Extended outlier inspection for 'Rank' ===
Description: Overall ranking, should be positive if present
Non null count      : 20293 / 24905
Min / Max           : 0.0 / 20104.0
Q1 / Q3             : 4885.0 / 15032.0
IQR                 : 10147.0
IQR envelope        : [-10335.5, 30252.5]
Suspected errors    : 187
Legit extremes      : 0

Sample suspected errors:


,anime_id,Name,Rank
19398,46294,Chokkan × Algorhythm♪ 3rd Season,0.0
21356,49513,Bucheon International Animation Festival 2021 ...,0.0
21425,49681,Vivy: Fluorite Eye’s Song - PILOT MOVIE,0.0
21479,49817,Fate/stay night Specials,0.0
21480,49818,Guimi Zhi Zhu,0.0



=== Extended outlier inspection for 'Popularity' ===
Description: Popularity ranking, should be positive if present
Non null count      : 24905 / 24905
Min / Max           : 0 / 24723
Q1 / Q3             : 6040.0 / 18491.0
IQR                 : 12451.0
IQR envelope        : [-12636.5, 37167.5]
Suspected errors    : 187
Legit extremes      : 0

Sample suspected errors:


,anime_id,Name,Popularity
19398,46294,Chokkan × Algorhythm♪ 3rd Season,0
21356,49513,Bucheon International Animation Festival 2021 ...,0
21425,49681,Vivy: Fluorite Eye’s Song - PILOT MOVIE,0
21479,49817,Fate/stay night Specials,0
21480,49818,Guimi Zhi Zhu,0


### 5.7 IQR based outlier analysis

We use the Interquartile Range (IQR) rule to identify extreme values for key numeric
variables. A value is considered an outlier if it lies outside:

[Q1 - 1.5 * IQR, Q3 + 1.5 * IQR].


In [23]:
def detect_outliers_iqr(series: pd.Series, factor: float = 1.5):
    """
    Detect outliers using the IQR method.

    Parameters
    ----------
    series : pd.Series
        Numeric series.
    factor : float
        IQR multiplier, typically 1.5 or 3.0.

    Returns
    -------
    lower_bound : float
    upper_bound : float
    mask_outlier : pd.Series[bool]
        Boolean mask indicating outliers.
    """
    s = series.dropna()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - factor * iqr
    upper_bound = q3 + factor * iqr

    mask_outlier = (series < lower_bound) | (series > upper_bound)

    return lower_bound, upper_bound, mask_outlier


In [24]:
# Outlier inspection for Score
if "Score" in df_typed.columns:
    lb_score, ub_score, mask_score_outlier = detect_outliers_iqr(df_typed["Score"], factor=1.5)

    n_total = df_typed["Score"].notna().sum()
    n_outliers = int(mask_score_outlier.sum())

    print("=== Outlier inspection - Score ===")
    print(f"IQR lower bound : {lb_score:.2f}")
    print(f"IQR upper bound : {ub_score:.2f}")
    print(f"Non-missing rows: {n_total}")
    print(f"Outlier count   : {n_outliers} ({n_outliers / n_total * 100:.2f} percent)")

    # Show extreme outliers
    score_outliers = df_typed.loc[mask_score_outlier, ["Name", "Score"]].sort_values("Score")
    print("\nExamples of lowest and highest Score outliers:")
    display(pd.concat([score_outliers.head(5), score_outliers.tail(5)]))

    # Plot Score highlighting outliers in Plotly
    fig_score = px.scatter(
        df_typed,
        x=np.arange(len(df_typed)),
        y="Score",
        color=mask_score_outlier.map({True: "Outlier", False: "Normal"}),
        labels={"x": "Index", "Score": "Score", "color": "Outlier flag"},
        title="Score values with IQR-based outlier flag",
    )
    fig_score.show()


=== Outlier inspection - Score ===
IQR lower bound : 3.74
IQR upper bound : 9.05
Non-missing rows: 15692
Outlier count   : 68 (0.43 percent)

Examples of lowest and highest Score outliers:


,Name,Score
2925,Tenkuu Danzai Skelter+Heaven,1.85
6947,Utsu Musume Sayuri,1.98
387,Hametsu no Mars,2.22
4194,Tsui no Sora,2.22
10984,Kokuhaku,2.30
4860,Pop,3.73
9880,Gintama°,9.06
5667,Steins;Gate,9.07
16617,Bleach: Sennen Kessen-hen,9.07
3961,Fullmetal Alchemist: Brotherhood,9.10


In [25]:
# Outlier inspection for Episodes
if "Episodes" in df_typed.columns:
    lb_eps, ub_eps, mask_eps_outlier = detect_outliers_iqr(df_typed["Episodes"], factor=1.5)

    n_total_eps = df_typed["Episodes"].notna().sum()
    n_outliers_eps = int(mask_eps_outlier.sum())

    print("=== Outlier inspection - Episodes ===")
    print(f"IQR lower bound : {lb_eps:.2f}")
    print(f"IQR upper bound : {ub_eps:.2f}")
    print(f"Non-missing rows: {n_total_eps}")
    print(f"Outlier count   : {n_outliers_eps} ({n_outliers_eps / n_total_eps * 100:.2f} percent)")

    eps_outliers = df_typed.loc[mask_eps_outlier, ["Name", "Type", "Episodes"]].sort_values("Episodes")
    print("\nExamples of lowest and highest Episodes outliers:")
    display(pd.concat([eps_outliers.head(5), eps_outliers.tail(5)]))

    fig_eps = px.scatter(
        df_typed,
        x=np.arange(len(df_typed)),
        y="Episodes",
        color=mask_eps_outlier.map({True: "Outlier", False: "Normal"}),
        labels={"x": "Index", "Episodes": "Episodes", "color": "Outlier flag"},
        title="Episodes values with IQR-based outlier flag",
    )
    fig_eps.update_yaxes(type="log")
    fig_eps.show()


=== Outlier inspection - Episodes ===
IQR lower bound : -17.00
IQR upper bound : 31.00
Non-missing rows: 24294
Outlier count   : 2856 (11.76 percent)

Examples of lowest and highest Episodes outliers:


,Name,Type,Episodes
20666,Xiao Shu Luo Fu Tiaozhan Ji,TV,32.0
5395,Biriken,TV,32.0
19866,Chongwu Wuyu,TV,32.0
11360,Shounen Ashibe: Go! Go! Goma-chan,TV,32.0
19844,Mobi Chuanshuo,TV,32.0
8881,Kirin Monoshiri Yakata,TV,1565.0
19404,Daomei Xiansheng,TV,1664.0
2266,Doraemon (1979),TV,1787.0
6723,Oyako Club,TV,1818.0
5974,Lan Mao,TV,3057.0


### 5.8 String data consistency: 

**5.8.1. Fuzzy matching for studios**

String fields such as Studios or Producers may contain near duplicates
due to inconsistent naming (for example, "Sunrise" vs "Sunrise Inc.").
We use fuzzy string matching to detect candidates for standardization.

In [26]:
try:
    from rapidfuzz import fuzz
except ImportError:
    import sys
    !{sys.executable} -m pip install rapidfuzz -q
    from rapidfuzz import fuzz

if "Studios" in df.columns:
    unique_studios_raw = df["Studios"].dropna().unique()
    studio_tokens = set()
    for s in unique_studios_raw:
        for part in str(s).split(","):
            token = part.strip()
            if token:
                studio_tokens.add(token)

    studio_list = sorted(studio_tokens)
    print(f"Total unique studio tokens: {len(studio_list)}")

    from itertools import combinations

    similar_pairs = []
    max_pairs = 500  # limit combinations for performance
    subset = studio_list[:max_pairs]

    for s1, s2 in combinations(subset, 2):
        score = fuzz.ratio(s1, s2)
        if score > 90 and s1 != s2:
            similar_pairs.append((s1, s2, score))

    if similar_pairs:
        similar_pairs.sort(key=lambda x: x[2], reverse=True)
        print("Potential duplicate / inconsistent studio names (similarity > 90):")
        for s1, s2, sc in similar_pairs[:20]:
            print(f"- '{s1}' vs '{s2}' (Similarity: {sc:.1f})")
    else:
        print("No highly similar studio name pairs found in the checked subset.")
else:
    print("Studios column not available.")


Total unique studio tokens: 1044
Potential duplicate / inconsistent studio names (similarity > 90):
- 'Chongzhuo Animation' vs 'Congzhuo Animation' (Similarity: 97.3)
- 'Asahi Production' vs 'Ashi Productions' (Similarity: 93.8)
- 'Gaina' vs 'Gainax' (Similarity: 90.9)
- 'ASK Animation Studio' vs 'Ai Si Animation Studio' (Similarity: 90.5)


**5.8.2. Data Format Listing**

In [27]:
import re

if "Aired" in df_typed.columns:
    s = df_typed["Aired"]
    mask_non_null = s.notna()

    # chỉ làm việc trên các giá trị không NaN
    aired_raw = s[mask_non_null].astype(str).str.strip()

    # Pattern 1: "Apr 3, 1998 to Apr 24, 1999"
    pat_range_full = re.compile(
        r"^[A-Z][a-z]{2} \d{1,2}, \d{4} to [A-Z][a-z]{2} \d{1,2}, \d{4}$"
    )

    # Pattern 2: "Oct 20, 1999 to ?"
    pat_range_open = re.compile(
        r"^[A-Z][a-z]{2} \d{1,2}, \d{4} to \?$"
    )

    # Pattern 3: "1-Sep-01"
    pat_single_dmy = re.compile(
        r"^\d{1,2}-[A-Za-z]{3}-\d{2}$"
    )

    # Pattern 4: "Sep 1, 2001"
    pat_single_full = re.compile(
        r"^[A-Z][a-z]{2} \d{1,2}, \d{4}$"
    )

    # Pattern 5: "Apr 1994 to Jan 27, 1995"
    pat_monYYYY_to_full = re.compile(
        r"^[A-Z][a-z]{2} \d{4} to [A-Z][a-z]{2} \d{1,2}, \d{4}$"
    )

    # Pattern 6: "2006"
    pat_year_only = re.compile(
        r"^\d{4}$"
    )

    # Pattern 7: "Jun 1984"
    pat_month_year = re.compile(
        r"^[A-Z][a-z]{2} \d{4}$"
    )

    # Pattern 8: "2005 to Feb 26, 2005"
    pat_year_to_full = re.compile(
        r"^\d{4} to [A-Z][a-z]{2} \d{1,2}, \d{4}$"
    )

    # Pattern 9: "Nov 27, 1998 to Jan 2000"
    pat_full_to_monYYYY = re.compile(
        r"^[A-Z][a-z]{2} \d{1,2}, \d{4} to [A-Z][a-z]{2} \d{4}$"
    )

    # Pattern 10: "2006 to 2007", "2000 to 2005", "1986 to 1987"
    pat_year_to_year = re.compile(
        r"^\d{4} to \d{4}$"
    )

    # Pattern 11: "Feb 2005 to Nov 2005", "Oct 1999 to Jun 2001"
    pat_monYYYY_to_monYYYY = re.compile(
        r"^[A-Z][a-z]{2} \d{4} to [A-Z][a-z]{2} \d{4}$"
    )

    # Pattern 12: "Aug 2008 to 2008"
    pat_monYYYY_to_year = re.compile(
        r"^[A-Z][a-z]{2} \d{4} to \d{4}$"
    )

    # Pattern 13: "Feb 21, 1997 to 2002"
    pat_full_to_year = re.compile(
        r"^[A-Z][a-z]{2} \d{1,2}, \d{4} to \d{4}$"
    )

    # Pattern 14 mới: "Jan 1917 to ?"
    pat_monYYYY_to_question = re.compile(
        r"^[A-Z][a-z]{2} \d{4} to \?$"
    )

    # Pattern 15 mới: "2006 to ?"
    pat_year_to_question = re.compile(
        r"^\d{4} to \?$"
    )

    # apply mask trên phần non null
    mask_range_full = aired_raw.str.match(pat_range_full)
    mask_range_open = aired_raw.str.match(pat_range_open)
    mask_single_dmy = aired_raw.str.match(pat_single_dmy)
    mask_single_full = aired_raw.str.match(pat_single_full)
    mask_monYYYY_to_full = aired_raw.str.match(pat_monYYYY_to_full)
    mask_year_only = aired_raw.str.match(pat_year_only)
    mask_month_year = aired_raw.str.match(pat_month_year)
    mask_year_to_full = aired_raw.str.match(pat_year_to_full)
    mask_full_to_monYYYY = aired_raw.str.match(pat_full_to_monYYYY)
    mask_year_to_year = aired_raw.str.match(pat_year_to_year)
    mask_monYYYY_to_monYYYY = aired_raw.str.match(pat_monYYYY_to_monYYYY)
    mask_monYYYY_to_year = aired_raw.str.match(pat_monYYYY_to_year)
    mask_full_to_year = aired_raw.str.match(pat_full_to_year)
    mask_monYYYY_to_question = aired_raw.str.match(pat_monYYYY_to_question)
    mask_year_to_question = aired_raw.str.match(pat_year_to_question)

    mask_known_pattern = (
        mask_range_full
        | mask_range_open
        | mask_single_dmy
        | mask_single_full
        | mask_monYYYY_to_full
        | mask_year_only
        | mask_month_year
        | mask_year_to_full
        | mask_full_to_monYYYY
        | mask_year_to_year
        | mask_monYYYY_to_monYYYY
        | mask_monYYYY_to_year
        | mask_full_to_year
        | mask_monYYYY_to_question
        | mask_year_to_question
    )
    mask_other = ~mask_known_pattern

    print("\n=== Aired format diagnostics ===")
    print(f"Total rows (non null only)                   : {len(aired_raw)}")
    print(f"'Mon D, YYYY to Mon D, YYYY'                 : {mask_range_full.sum()}")
    print(f"'Mon D, YYYY to ?'                           : {mask_range_open.sum()}")
    print(f"'D-Mon-YY'                                   : {mask_single_dmy.sum()}")
    print(f"'Mon D, YYYY' (single date)                  : {mask_single_full.sum()}")
    print(f"'Mon YYYY to Mon D, YYYY'                    : {mask_monYYYY_to_full.sum()}")
    print(f"'YYYY' (year only)                           : {mask_year_only.sum()}")
    print(f"'Mon YYYY'                                   : {mask_month_year.sum()}")
    print(f"'YYYY to Mon D, YYYY'                        : {mask_year_to_full.sum()}")
    print(f"'Mon D, YYYY to Mon YYYY'                    : {mask_full_to_monYYYY.sum()}")
    print(f"'YYYY to YYYY'                               : {mask_year_to_year.sum()}")
    print(f"'Mon YYYY to Mon YYYY'                       : {mask_monYYYY_to_monYYYY.sum()}")
    print(f"'Mon YYYY to YYYY'                           : {mask_monYYYY_to_year.sum()}")
    print(f"'Mon D, YYYY to YYYY'                        : {mask_full_to_year.sum()}")
    print(f"'Mon YYYY to ?'                              : {mask_monYYYY_to_question.sum()}")
    print(f"'YYYY to ?'                                  : {mask_year_to_question.sum()}")
    print(f"Other / unexpected format                    : {mask_other.sum()}")

    id_cols = [c for c in ["anime_id", "Name", "name"] if c in df_typed.columns]
    show_cols = id_cols + ["Aired"] if id_cols else ["Aired"]

    if mask_other.any():
        print("\nExamples - unexpected Aired formats (non null only):")
        # lấy index thực tế trên df_typed
        idx_other = aired_raw[mask_other].index
        display(df_typed.loc[idx_other, show_cols].head(10))
else:
    print("[Aired diagnostics] Column 'Aired' not found, skipping.")



=== Aired format diagnostics ===
Total rows (non null only)                   : 23990
'Mon D, YYYY to Mon D, YYYY'                 : 9317
'Mon D, YYYY to ?'                           : 824
'D-Mon-YY'                                   : 0
'Mon D, YYYY' (single date)                  : 9786
'Mon YYYY to Mon D, YYYY'                    : 5
'YYYY' (year only)                           : 2229
'Mon YYYY'                                   : 320
'YYYY to Mon D, YYYY'                        : 15
'Mon D, YYYY to Mon YYYY'                    : 36
'YYYY to YYYY'                               : 88
'Mon YYYY to Mon YYYY'                       : 118
'Mon YYYY to YYYY'                           : 27
'Mon D, YYYY to YYYY'                        : 127
'Mon YYYY to ?'                              : 190
'YYYY to ?'                                  : 908
Other / unexpected format                    : 0


In [28]:
import re

# ========= Premiered format diagnostics =========
if "Premiered" in df_typed.columns:
    s = df_typed["Premiered"]
    mask_non_null = s.notna()
    prem_raw = s[mask_non_null].astype(str).str.strip()

    # Season-year, ví dụ: "spring 1998", "Fall 2004"
    pat_season_year = re.compile(r"^(spring|summer|fall|winter) \d{4}$", flags=re.IGNORECASE)

    # Year-only: "2003", "1997", "1995", "2006"
    pat_year_only = re.compile(r"^\d{4}$")

    # Month-year: "Jun 1984"
    pat_month_year = re.compile(r"^[A-Z][a-z]{2} \d{4}$")

    mask_season_year = prem_raw.str.match(pat_season_year)
    mask_year_only = prem_raw.str.match(pat_year_only)
    mask_month_year = prem_raw.str.match(pat_month_year)
    mask_unknown = prem_raw.str.upper().eq("UNKNOWN")
    mask_empty = prem_raw.eq("")

    mask_known_pattern = (
        mask_season_year
        | mask_year_only
        | mask_month_year
        | mask_unknown
        | mask_empty
    )
    mask_other = ~mask_known_pattern

    print("\n=== Premiered format diagnostics ===")
    print(f"Total rows (non null only)   : {len(prem_raw)}")
    print(f"Pattern 'season YYYY'        : {mask_season_year.sum()}")
    print(f"Pattern 'YYYY' (year only)   : {mask_year_only.sum()}")
    print(f"Pattern 'Mon YYYY'           : {mask_month_year.sum()}")
    print(f"Empty string                 : {mask_empty.sum()}")
    print(f"Other / unexpected format    : {mask_other.sum()}")

    id_cols = [c for c in ["anime_id", "Name", "name"] if c in df_typed.columns]
    show_cols = id_cols + ["Premiered"] if id_cols else ["Premiered"]

    if mask_other.any():
        print("\nExamples - unexpected Premiered formats:")
        idx_other = prem_raw[mask_other].index
        display(df_typed.loc[idx_other, show_cols].head(10))
else:
    print("[Premiered diagnostics] Column 'Premiered' not found, skipping.")



=== Premiered format diagnostics ===
Total rows (non null only)   : 5506
Pattern 'season YYYY'        : 5506
Pattern 'YYYY' (year only)   : 0
Pattern 'Mon YYYY'           : 0
Empty string                 : 0
Other / unexpected format    : 0


In [59]:
if "Duration" in df_typed.columns:
    s_raw = df_typed["Duration"]
    mask_non_null = s_raw.notna()
    s = s_raw[mask_non_null].astype(str).str.strip()

    # Patterns hợp lệ
    pat_min_per_ep = re.compile(r"^\d+ min per ep$")            # 24 min per ep
    pat_hr_min_per_ep = re.compile(r"^\d+ hr \d+ min per ep$")  # 1 hr 30 min per ep
    pat_hr_per_ep = re.compile(r"^\d+ hr per ep$")              # 1 hr per ep
    pat_sec_per_ep = re.compile(r"^\d+ sec per ep$")            # 30 sec per ep
    pat_hr_min = re.compile(r"^\d+ hr \d+ min$")                # 1 hr 55 min
    pat_hr_only = re.compile(r"^\d+ hr$")                       # 2 hr
    pat_min_only = re.compile(r"^\d+ min$")                     # 24 min
    pat_sec_only = re.compile(r"^\d+ sec$")                     # 12 sec

    mask_min_per_ep = s.str.match(pat_min_per_ep)
    mask_hr_min_per_ep = s.str.match(pat_hr_min_per_ep)
    mask_hr_per_ep = s.str.match(pat_hr_per_ep)
    mask_sec_per_ep = s.str.match(pat_sec_per_ep)
    mask_hr_min = s.str.match(pat_hr_min)
    mask_hr_only = s.str.match(pat_hr_only)
    mask_min_only = s.str.match(pat_min_only)
    mask_sec_only = s.str.match(pat_sec_only)

    mask_known = (
        mask_min_per_ep
        | mask_hr_min_per_ep
        | mask_hr_per_ep
        | mask_sec_per_ep
        | mask_hr_min
        | mask_hr_only
        | mask_min_only
        | mask_sec_only
    )

    mask_unknown = ~mask_known

    print("\n=== Duration format diagnostics (EDA only) ===")
    print(f"Non null rows:                     {len(s)}")
    print(f"'N min per ep'                    : {mask_min_per_ep.sum()}")
    print(f"'N hr M min per ep'               : {mask_hr_min_per_ep.sum()}")
    print(f"'N hr per ep'                     : {mask_hr_per_ep.sum()}")
    print(f"'N sec per ep'                    : {mask_sec_per_ep.sum()}")
    print(f"'N hr M min'                      : {mask_hr_min.sum()}")
    print(f"'N hr'                            : {mask_hr_only.sum()}")
    print(f"'N min'                           : {mask_min_only.sum()}")
    print(f"'N sec'                           : {mask_sec_only.sum()}")
    print(f"Unexpected duration format        : {mask_unknown.sum()}")

    id_cols = [c for c in ["anime_id", "Name"] if c in df_typed.columns]
    show_cols = id_cols + ["Duration"] if id_cols else ["Duration"]

    if mask_unknown.any():
        print("\nExamples  unexpected Duration formats (non null only):")
        idx_unknown = s[mask_unknown].index
        display(df_typed.loc[idx_unknown, show_cols].head(10))
else:
    print("[Duration diagnostics] Column 'Duration' not found.")



=== Duration format diagnostics (EDA only) ===
Non null rows:                     24242
'N min per ep'                    : 12309
'N hr M min per ep'               : 28
'N hr per ep'                     : 6
'N sec per ep'                    : 294
'N hr M min'                      : 1988
'N hr'                            : 117
'N min'                           : 9189
'N sec'                           : 311
Unexpected duration format        : 0


In [ ]:
if "Source" in df_typed.columns:

    # Accepted vocabulary
    source_vocab = {
        "original", "manga", "light novel", "visual novel", "game",
        "4-koma manga", "doujinshi", "novel", "other", "book",
        "card game", "radio", "web manga", "picture book", "music", "mixed media",
        "web novel"
    }

    # Only check non-null values
    s_raw = df_typed["Source"]
    mask_non_null = s_raw.notna()
    s = s_raw[mask_non_null].astype(str).str.strip()
    s_norm = s.str.lower()

    mask_in_vocab = s_norm.isin(source_vocab)
    mask_unknown = ~mask_in_vocab

    print("\n=== Source vocabulary diagnostics (EDA only) ===")
    print(f"Non null rows:               {len(s)}")
    print(f"Valid source values:         {mask_in_vocab.sum()}")
    print(f"Unexpected source values:    {mask_unknown.sum()}")

    id_cols = [c for c in ["anime_id", "Name"] if c in df_typed.columns]
    show_cols = id_cols + ["Source"] if id_cols else ["Source"]

    if mask_unknown.any():
        print("\nExamples – unexpected Source values:")
        idx_unknown = s[mask_unknown].index
        display(df_typed.loc[idx_unknown, show_cols].head(10))

else:
    print("[Source diagnostics] Column 'Source' not found.")



=== Source vocabulary diagnostics (EDA only) ===
Non null rows:               21216
Valid source values:         21216
Unexpected source values:    0


| EDA Finding                                  | Details                                                                                           |
|----------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Total records with Source**                | 21,216                                                                                            |
| **Data cleanliness**                         | `Perfect` – zero typos, zero weird values, fully standardized                                    |
| **Input method evidence**                    | `Likely dropdown` – explains the exceptional consistency                                          |
| **Label type**                               | `Pure single-label categorical` – each anime has exactly one source                               |
| **Text normalization needed**                | `None` – no case, spacing or spelling issues                                                      |
| **Final takeaway**                           | `Cleanest column in the dataset` • ready-to-use as-is • simple one-hot or label encoding sufficient |

**5.8.3. Scraping Error Detection for List-String Columns**

In [ ]:
def split_tokens(value, sep=","):
    if pd.isna(value):
        return []
    return [t.strip() for t in str(value).split(sep) if t.strip() != ""]


def run_list_string_diagnostics(df, col, high_token_threshold=10,
                                high_char_len_threshold=70,
                                max_delim_count_for_suspicious=1):
    """
    Generic EDA diagnostics cho cột dạng 'list-string' phân tách bằng dấu phẩy.
    Áp dụng được cho: Genres, Producers, Licensors, Studios.
    """

    if col not in df.columns:
        print(f"[Diagnostics] Column '{col}' not found, skipping.")
        return

    s_raw = df[col]
    mask_non_null = s_raw.notna()
    s = s_raw[mask_non_null].astype(str).str.strip()

    # Token lists + counts
    token_lists = s.apply(split_tokens)
    token_counts = token_lists.apply(len)

    # High token threshold
    mask_many_tokens = token_counts > high_token_threshold

    # Delimiter check
    contains_comma = s.str.contains(",", regex=False)
    contains_semicolon = s.str.contains(";", regex=False)
    mask_no_delim = ~contains_comma & ~contains_semicolon

    # Text length vs delimiter count
    char_len = s.str.len()
    delim_count = s.str.count(",") + s.str.count(";")

    mask_long_few_delims = (
        (char_len >= high_char_len_threshold)
        & (delim_count <= max_delim_count_for_suspicious)
    )

    # Print summary
    print(f"\n=== Diagnostics for '{col}' (list-string EDA) ===")
    print(f"Total rows                         : {len(s_raw)}")
    print(f"Non null entries                   : {mask_non_null.sum()}")
    print(f"Rows with > {high_token_threshold} tokens  : {mask_many_tokens.sum()}")
    print(f"Rows without ',' or ';'            : {mask_no_delim.sum()}")
    print(
        f"Rows long text but few delimiters  : {mask_long_few_delims.sum()} "
        f"(len >= {high_char_len_threshold}, delimiters <= {max_delim_count_for_suspicious})"
    )

    id_cols = [c for c in ["anime_id", "Name"] if c in df.columns]
    show_cols = id_cols + [col] if id_cols else [col]

    # Examples
    if mask_many_tokens.any():
        print(f"\nExamples  many tokens in '{col}' (> {high_token_threshold} tokens):")
        idx_many = token_counts[mask_many_tokens].index
        display(df.loc[idx_many, show_cols].assign(_token_count=token_counts.loc[idx_many]).head(5))

    if mask_no_delim.any():
        print(f"\nExamples  no delimiter ',' or ';' in '{col}':")
        idx_no = s[mask_no_delim].index
        display(df.loc[idx_no, show_cols].head(5))

    if mask_long_few_delims.any():
        print(f"\nExamples  long text but few delimiters in '{col}':")
        idx_long = s[mask_long_few_delims].index
        display(
            df.loc[idx_long, show_cols]
              .assign(_char_len=char_len.loc[idx_long],
                      _delim_count=delim_count.loc[idx_long])
              .head(5)
        )

In [55]:
run_list_string_diagnostics(df_typed, "Genres")


=== Diagnostics for 'Genres' (list-string EDA) ===
Total rows                         : 24905
Non null entries                   : 19976
Rows with > 10 tokens  : 0
Rows without ',' or ';'            : 8859
Rows long text but few delimiters  : 0 (len >= 70, delimiters <= 1)

Examples  no delimiter ',' or ';' in 'Genres':


,anime_id,Name,Genres
5,15,Eyeshield 21,Sports
12,22,Tennis no Ouji-sama,Sports
27,46,Rurouni Kenshin: Meiji Kenkaku Romantan - Ishi...,Drama
53,73,Full Metal Panic! The Second Raid,Action
93,114,Sakigake!! Cromartie Koukou,Comedy


| EDA Finding                                  | Details                                                                                           |
|----------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Total records with Genres**                | 19,976                                                                                            |
| **Single-genre ratio**                       | `~44%` (8,859 rows) – no comma                                                                    |
| **Multi-genre ratio**                        | `~56%` – majority of anime combine multiple genres (e.g. Action, Comedy, Romance)                |
| **Multi-label severity**                     | `Very high` – taking only first genre will severely distort meaning (Comedy, Horror → becomes just Comedy) |
| **Text quality**                             | `Extremely clean` – all short, standard keywords, 0 rows >10 tokens, no formatting errors        |
| **Final takeaway**                           | `True multi-label field` • must explode / multi-hot encode • never simplify to first genre only    |

In [56]:
run_list_string_diagnostics(df_typed, "Producers")


=== Diagnostics for 'Producers' (list-string EDA) ===
Total rows                         : 24905
Non null entries                   : 11555
Rows with > 10 tokens  : 66
Rows without ',' or ';'            : 6622
Rows long text but few delimiters  : 1 (len >= 70, delimiters <= 1)

Examples  many tokens in 'Producers' (> 10 tokens):


,anime_id,Name,Producers,_token_count
213,237,Koukyoushihen Eureka Seven,"Aniplex, Bandai Visual, Kadokawa Shoten, Maini...",11
2800,3089,The Sky Crawlers,"Bandai Visual, VAP, Yomiuri Telecasting, d-rig...",14
5588,9000,Toaru Hikuushi e no Tsuioku,"TV Tokyo, Bandai Visual, TMS Entertainment, Me...",12
6175,10389,Momo e no Tegami,"Bandai Visual, OLM, Kadokawa Shoten, HoriPro, ...",13
6934,13335,Fuse: Teppou Musume no Torimonochou,"TV Tokyo, Sotsu, MediaNet, Kids Station, Nippo...",12



Examples  no delimiter ',' or ';' in 'Producers':


,anime_id,Name,Producers
0,1,Cowboy Bebop,Bandai Visual
2,6,Trigun,Victor Entertainment
25,44,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...,Aniplex
37,56,Avenger,Yomiko Advertising
42,61,D.N.Angel,Kadokawa Shoten



Examples  long text but few delimiters in 'Producers':


,anime_id,Name,Producers,_char_len,_delim_count
8262,20469,Himeyuri,"Himeyuri Alumnae Incorporated Foundation, ASIA...",70,1


| EDA Finding                                  | Details                                                                                           |
|----------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Multi-label intensity**                    | `High`     | ~43% records have multiple producers • max = 14 (The Sky Crawlers)                                |
| **Collaboration nature**                     | `Very strong` – reflects real anime “Production Committee” model (many companies co-fund)        |
| **Single-value ratio**                       | Only ~57% are single producer • far lower than Studios or Licensors                               |
| **Risk of simplification**                   | `High risk` – taking only “first producer” will lose critical signal (Aniplex, TV Tokyo, etc. can appear anywhere in the list) |
| **Final takeaway**                           | `Must treat as true multi-label` • explode / multi-hot encoding is mandatory • cannot simplify like Studios |

In [57]:
run_list_string_diagnostics(df_typed, "Licensors")


=== Diagnostics for 'Licensors' (list-string EDA) ===
Total rows                         : 24905
Non null entries                   : 4735
Rows with > 10 tokens  : 0
Rows without ',' or ';'            : 3865
Rows long text but few delimiters  : 0 (len >= 70, delimiters <= 1)

Examples  no delimiter ',' or ';' in 'Licensors':


,anime_id,Name,Licensors
1,5,Cowboy Bebop: Tengoku no Tobira,Sony Pictures Entertainment
4,8,Bouken Ou Beet,Illumitoon Entertainment
8,18,Initial D Fourth Stage,Funimation
9,19,Monster,VIZ Media
10,20,Naruto,VIZ Media


| ⚠️ EDA Finding                                  | Details                                                                                           |
|------------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Structural complexity**                      | `**Very low complexity**` – ~81.6% (3,865/4,735) are single licensor (no comma)                  |
| **Multi-licensor cases**                       | Only ~18.4% contain commas → far simpler than Genres/Producers                                   |
| **Text cleanliness**                           | `**Extremely clean**` – 0 rows with >10 tokens, 0 rows with long text but few delimiters         |
| **Example values**                             | Sony Pictures Entertainment, Funimation, VIZ Media → consistent, well-formatted company names       |
| **Final takeaway**                             | `**Cleanest multi-label column so far**`<br>Easy to parse, minimal cleaning needed, safe to explode directly into multi-hot or use as-is |

In [58]:
run_list_string_diagnostics(df_typed, "Studios")


=== Diagnostics for 'Studios' (list-string EDA) ===
Total rows                         : 24905
Non null entries                   : 14379
Rows with > 10 tokens  : 0
Rows without ',' or ';'            : 13354
Rows long text but few delimiters  : 0 (len >= 70, delimiters <= 1)

Examples  no delimiter ',' or ';' in 'Studios':


,anime_id,Name,Studios
0,1,Cowboy Bebop,Sunrise
1,5,Cowboy Bebop: Tengoku no Tobira,Bones
2,6,Trigun,Madhouse
3,7,Witch Hunter Robin,Sunrise
4,8,Bouken Ou Beet,Toei Animation


| ⚠️ EDA Finding                                  | Details                                                                                           |
|------------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Cardinality**                                | **High cardinality**– thousands of unique studio names                                        |
| **Structural complexity**                      | **Low complexity** – ~93% single studio, only ~7% contain commas                                |                               |
| **Multi-studio records**                       |`**~7% collaborations** → require explode or multi-hot encoding                                  |
| **Name duplication & variants**                | **Heavy duplication**` (e.g. “Toei Animation” vs “Toei”, “J.C.Staff” vs “J.C. Staff”) → must canonicalize |
| **Final takeaway**                             | **Easy to parse, extremely hard to standardize**`<br>Mandatory: impute missing → deduplicate names → handle multi-studio before encoding |

### 5.9 Business rule and logical validation



### 5.9.1 Aired start vs end date

Check entries where `Aired` is in the form `"start to end"` but the parsed end date is earlier than the start date (`end < start`).

This is a sign of temporal inconsistency, typically caused by scraping errors or swapped start/end values.


In [62]:
if "Aired" in df_typed.columns:
    s = df_typed["Aired"].astype(str).str.strip()

    # Tách "start to end"
    aired_split = s.str.split(" to ", n=1, expand=True)

    start_raw = aired_split[0]
    end_raw = aired_split[1] if aired_split.shape[1] > 1 else None

    start_dates = pd.to_datetime(start_raw, errors="coerce")
    end_dates = pd.to_datetime(end_raw, errors="coerce") if end_raw is not None else None

    if end_raw is not None:
        invalid_dates_mask = (
            start_dates.notna()
            & end_dates.notna()
            & (end_dates < start_dates)
        )

        print("\n=== 5.9.1 Aired start vs end date ===")
        print(f"Total rows                    : {len(s)}")
        print(f"Rows with start and end dates : {(start_dates.notna() & end_dates.notna()).sum()}")
        print(f"Rows with end < start         : {invalid_dates_mask.sum()}")

        if invalid_dates_mask.any():
            id_cols = [c for c in ["anime_id", "Name"] if c in df_typed.columns]
            show_cols = id_cols + ["Aired", "Score", "Type", "Status"] if id_cols else ["Aired", "Score", "Type", "Status"]

            print("\nExamples - Aired end date before start date:")
            display(df_typed.loc[invalid_dates_mask, show_cols].head(10))
    else:
        print("[5.9.1] No ' to ' pattern found in Aired, or no end part to compare.")
else:
    print("[5.9.1] Column 'Aired' not found, skipping.")



=== 5.9.1 Aired start vs end date ===
Total rows                    : 24905
Rows with start and end dates : 9317
Rows with end < start         : 7

Examples - Aired end date before start date:


,anime_id,Name,Aired,Score,Type,Status
3223,3705,Flashback Game,"Oct 18, 2001 to Apr 18, 2001",5.28,OVA,Finished Airing
11672,33126,Kurogane no Linebarrels Picture Drama,"Dec 14, 2009 to Aug 26, 2009",5.97,Special,Finished Airing
12251,34234,Ohayou! Kokekkou-san,"Oct 2, 2016 to Sep 17, 2016",NaN,TV,Finished Airing
14896,38571,Tobidase! Dokan-kun,"Oct 26, 2011 to Mar 14, 2011",NaN,TV,Finished Airing
17838,43976,Qi Jiguang Yingxiong Chuan,"Jun 1, 2012 to May 1, 2012",NaN,Movie,Finished Airing
21733,50258,Tesla Note: Mickey to Oliver no Agent Yousei K...,"Sep 29, 2021 to Jan 16, 2021",5.35,ONA,Finished Airing
24363,55038,Flow Machines,"Sep 8, 2021 to Aug 8, 2021",NaN,ONA,Finished Airing


|⚠️ EDA Finding                                  | Details                                                                                           |
|------------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Scope of detection**                         | Only **7 End < Start** cases found — limited to **9,317 records** with full format `Mon DD, YYYY to Mon DD YYYY` |
| **Critical hidden risk**                       | Remaining records (especially **88 "YYYY to YYYY"** rows + other coarse formats) **cannot be validated** → high chance of undetected date inversions |
| **Format complexity**                          | `Aired` column has **≥12 different formats** → **not usable as-is**                               |
| **Final takeaway**                             | **7 errors are just the tip of the iceberg** — full parsing & standardization of `Aired` is mandatory before any date logic |

In [64]:
cols_needed = {"Score", "Scored By", "Members"}
if cols_needed.issubset(df_typed.columns):

    score_raw = df_typed["Score"]
    scored_by_raw = df_typed["Scored By"]
    members_raw = df_typed["Members"]

    score_num = pd.to_numeric(score_raw, errors="coerce")
    scored_by_num = pd.to_numeric(scored_by_raw, errors="coerce")
    members_num = pd.to_numeric(members_raw, errors="coerce")

    # Rule 1: Scored By > Members
    mask_scored_by_gt_members = scored_by_num > members_num

    # Rule 2: Score not null but Scored By == 0 or NaN
    mask_score_not_null = score_num.notna()
    mask_scored_by_zero_or_na = scored_by_num.isna() | (scored_by_num == 0)
    mask_score_without_voters = mask_score_not_null & mask_scored_by_zero_or_na

    # Optional Rank check
    mask_rank_issue = pd.Series(False, index=df_typed.index)
    if "Rank" in df_typed.columns:
        rank_raw = df_typed["Rank"]
        rank_num = pd.to_numeric(rank_raw, errors="coerce")
        mask_rank_not_null = rank_num.notna()
        mask_score_null = score_num.isna()
        mask_rank_issue = mask_rank_not_null & mask_score_null

    print("\n=== 5.9.2 Score - Scored By - Members - Rank ===")
    print(f"Total rows                              : {len(df_typed)}")
    print(f"Rows where 'Scored By' > 'Members'      : {mask_scored_by_gt_members.sum()}")
    print(f"Rows where Score not null but Scored By == 0/NaN : {mask_score_without_voters.sum()}")
    if "Rank" in df_typed.columns:
        print(f"Rows where Rank not null but Score null : {mask_rank_issue.sum()}")

    id_cols = [c for c in ["anime_id", "Name"] if c in df_typed.columns]
    show_cols_score = id_cols + ["Score", "Members", "Scored By"] if id_cols else ["Score", "Members", "Scored By"]

    if mask_scored_by_gt_members.any():
        print("\nExamples - 'Scored By' > 'Members':")
        display(df_typed.loc[mask_scored_by_gt_members, show_cols_score].head(10))

    if mask_score_without_voters.any():
        print("\nExamples - Score not null but Scored By == 0/NaN:")
        display(df_typed.loc[mask_score_without_voters, show_cols_score].head(10))

    if "Rank" in df_typed.columns and mask_rank_issue.any():
        show_cols_rank = id_cols + ["Rank", "Score"] if id_cols else ["Rank", "Score"]
        print("\nExamples - Rank not null but Score null:")
        display(df_typed.loc[mask_rank_issue, show_cols_rank].head(10))
else:
    print("[5.9.2] Missing one of Score / Scored By / Members, skipping.")



=== 5.9.2 Score - Scored By - Members - Rank ===
Total rows                              : 24905
Rows where 'Scored By' > 'Members'      : 0
Rows where Score not null but Scored By == 0/NaN : 0
Rows where Rank not null but Score null : 7405

Examples - Rank not null but Score null:


,anime_id,Name,Rank,Score
1578,1739,Shibawanko no Wa no Kokoro,19660.0,NaN
1699,1863,Silk Road Shounen Yuuto,19912.0,NaN
2476,2701,Susie-chan to Marvy,12706.0,NaN
2483,2708,Wankorobee,13327.0,NaN
2508,2735,Mugen Senki Portriss,18320.0,NaN
2515,2742,Omoikkiri Kagaku Adventure Sou Nanda!,18679.0,NaN
2523,2750,Heisei Inu Monogatari Bow Movie,16386.0,NaN
2667,2930,Tensai? Dr. Hamax,12881.0,NaN
2786,3073,Robby to Kerobby,19200.0,NaN
2848,3164,Ichigo Ichie: Koibana Tomobana,16693.0,NaN


| ⚠️ EDA Finding      | DETAILS                                                                                                             |
|------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------|
| **Scored By > Members** (0 cases)               | Fully consistent → no anomalies detected                                                                            |
| **Score exists but Scored By = 0 or NaN** (0 cases)  | Perfectly consistent → Score always backed by at least one vote                                                    |
| **Rank exists but Score = NaN** (7,405 cases ≈ 30%)  | **Critical & widespread data error**<br>Anime cannot have a rank without a score<br>→ These ranks (often >15,000–20,000) are placeholder values from source system (likely MAL) for titles with insufficient votes |
| **Final Conclusion – Scoring Section**               | • Score ↔ Scored By ↔ Members relationships are **100% reliable**<br>• Rank column is **heavily contaminated** (~30% records) with meaningless placeholder ranks<br>• Rank must be treated with extreme caution or rebuilt only from valid Scores in later stages |

### 5.9.3 Episodes vs Status, Type vs Episodes, Duration vs Episodes

Check structural consistency between `Episodes`, `Status`, `Type`, and `Duration` using simple heuristics:

- `Status = Finished Airing` but `Episodes` is 0 or missing.
- `Type = Movie` but `Episodes > 1`.
- `Episodes == 1` but `Duration` contains `"per ep"`, which usually describes per-episode runtime for multi-episode shows.

These are heuristics to detect potential data mismatches rather than strict business rules.


In [66]:
cols_needed = {"Episodes", "Status", "Type", "Duration"}
if cols_needed.issubset(df_typed.columns):

    episodes_raw = df_typed["Episodes"]
    ep_num = pd.to_numeric(episodes_raw, errors="coerce")

    status_raw = df_typed["Status"].astype(str).str.strip()
    type_raw = df_typed["Type"].astype(str).str.strip()
    duration_raw = df_typed["Duration"].astype(str).str.strip()

    # Rule 1: Finished Airing but Episodes <= 0 or NaN
    mask_finished = status_raw.str.lower().eq("finished airing")
    mask_ep_missing_or_zero = ep_num.isna() | (ep_num <= 0)
    mask_finished_bad_episodes = mask_finished & mask_ep_missing_or_zero

    # Rule 2: Type = Movie but Episodes > 1
    mask_movie = type_raw.str.lower().eq("movie")
    mask_movie_many_eps = mask_movie & (ep_num > 1)

    # Rule 3: Episodes == 1 but Duration string suggests "per ep"
    mask_ep_one = ep_num == 1
    mask_duration_per_ep = duration_raw.str.contains("per ep", case=False, na=False)
    mask_single_with_per_ep = mask_ep_one & mask_duration_per_ep

    print("\n=== 5.9.3 Episodes, Status, Type, Duration ===")
    print(f"Total rows                                      : {len(df_typed)}")
    print(f"Finished Airing but Episodes <= 0/NaN          : {mask_finished_bad_episodes.sum()}")
    print(f"Type = Movie but Episodes > 1                  : {mask_movie_many_eps.sum()}")
    print(f"Episodes == 1 but Duration contains 'per ep'   : {mask_single_with_per_ep.sum()}")

    id_cols = [c for c in ["anime_id", "Name"] if c in df_typed.columns]
    base_cols = ["Episodes", "Status", "Type", "Duration"]
    show_cols = id_cols + base_cols if id_cols else base_cols

    if mask_finished_bad_episodes.any():
        print("\nExamples - Finished Airing but Episodes <= 0/NaN:")
        display(df_typed.loc[mask_finished_bad_episodes, show_cols].head(10))

    if mask_movie_many_eps.any():
        print("\nExamples - Type = Movie but Episodes > 1:")
        display(df_typed.loc[mask_movie_many_eps, show_cols].head(10))

    if mask_single_with_per_ep.any():
        print("\nExamples - Episodes == 1 but Duration looks 'per ep':")
        display(df_typed.loc[mask_single_with_per_ep, show_cols].head(10))
else:
    print("[5.9.3] Some of Episodes/Status/Type/Duration missing, skipping.")



=== 5.9.3 Episodes, Status, Type, Duration ===
Total rows                                      : 24905
Finished Airing but Episodes <= 0/NaN          : 9
Type = Movie but Episodes > 1                  : 96
Episodes == 1 but Duration contains 'per ep'   : 0

Examples - Finished Airing but Episodes <= 0/NaN:


,anime_id,Name,Episodes,Status,Type,Duration
9163,24775,Koishite!! Namashi-chan,NaN,Finished Airing,TV,1 min
10330,30129,Konna Ko Iru kana,NaN,Finished Airing,TV,1 min
11231,32237,Burutabu-chan,NaN,Finished Airing,TV,10 sec
13921,37024,Momoya Norihei Anime CM,NaN,Finished Airing,Special,30 sec
20800,48258,Super Talk Talk,NaN,Finished Airing,TV,14 min
24890,55721,See You on Our Veranda,NaN,Finished Airing,ONA,6 min
24891,55722,No More Princes,NaN,Finished Airing,ONA,5 min
24892,55723,The Forgotten Princess Just Wants Peace,NaN,Finished Airing,ONA,4 min
24895,55726,"Die, Please!",NaN,Finished Airing,ONA,5 min



Examples - Type = Movie but Episodes > 1:


,anime_id,Name,Episodes,Status,Type,Duration
1327,1462,Memories,3.0,Finished Airing,Movie,37 min per ep
1535,1689,Byousoku 5 Centimeter,3.0,Finished Airing,Movie,22 min per ep
1742,1911,Top wo Nerae! & Top wo Nerae 2! Gattai Movie!!,2.0,Finished Airing,Movie,1 hr 38 min per ep
1778,1951,Manie-Manie: Meikyuu Monogatari,3.0,Finished Airing,Movie,16 min per ep
2397,2611,Panda Kopanda,2.0,Finished Airing,Movie,36 min per ep
2506,2733,MiniPato,3.0,Finished Airing,Movie,12 min per ep
2695,2962,Digimon Adventure 02 Movies,2.0,Finished Airing,Movie,32 min per ep
3078,3508,Genius Party,7.0,Finished Airing,Movie,14 min per ep
4488,6189,Baton,3.0,Finished Airing,Movie,20 min per ep
4738,6795,Genius Party Beyond,5.0,Finished Airing,Movie,16 min per ep


| ⚠️ EDA Finding                         | Details                                                                                                           |
|------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------|
| **Status = Finished but Episodes = NaN** (9 cases)   | Completed anime with missing episode count → all are ultra-short (10s–14min), likely single episodes mislabeled  |
| **Type = Movie but Episodes > 1** (96 cases)         | **Most critical issue**: Movies must have exactly 1 episode. 96 records have 2–7 → clear logic violation          |
| **Episodes = 1 but Duration has "per ep."** (0 cases)| No issues found → consistent                                                                                      |
| **Rating = G but contains Ecchi genre** (several)    | **Severe contradiction**: "All Ages" vs adult content → definite data error                                     |
| **Final Conclusion**                                 | Dataset has **multiple critical logic conflicts** in:<br>• Status ↔ Episodes<br>• Type ↔ Episodes<br>• Rating ↔ Genres<br>**Must be fixed before modeling** |

### 5.9.4 Rating vs Genres

Soft semantic consistency check between `Rating` and `Genres`:

- Titles with a "light" rating (G, PG, PG-13 and their common MAL variants) but whose `Genres` contain `Hentai` or `Ecchi`.

This is not a hard rule, but a way to flag potentially inconsistent content labels for manual review.


In [ ]:
if {"Rating", "Genres"}.issubset(df_typed.columns):

    rating_raw = df_typed["Rating"].astype(str).str.strip()
    genres_raw = df_typed["Genres"].astype(str).str.strip()

    rating_lower = rating_raw.str.lower()
    genres_lower = genres_raw.str.lower()

    # Define all explicit "light rating" variants we accept as true light ratings
    LIGHT_RATING_SET = {
        "g",
        "g - all ages",
        "pg",
        "pg - children",
        "pg-13",
    }

    mask_light_rating = rating_lower.isin(LIGHT_RATING_SET)

    # Genres contains Hentai or Ecchi
    mask_hentai_ecchi = (
        genres_lower.str.contains("hentai", na=False)
        | genres_lower.str.contains("ecchi", na=False)
    )

    mask_suspicious = mask_light_rating & mask_hentai_ecchi

    print("\n=== 5.9.4 Rating vs Genres ===")
    print(f"Total rows                                : {len(df_typed)}")
    print(f"Rows with light rating                    : {mask_light_rating.sum()}")
    print(f"Rows where Genres contains Hentai/Ecchi   : {mask_hentai_ecchi.sum()}")
    print(f"Suspicious rows (light rating & H/E)      : {mask_suspicious.sum()}")

    if mask_suspicious.any():
        id_cols = [c for c in ["anime_id", "Name"] if c in df_typed.columns]
        show_cols = id_cols + ["Rating", "Genres"] if id_cols else ["Rating", "Genres"]

        print("\nExamples - light rating but Hentai/Ecchi in Genres:")
        idx_susp = mask_suspicious[mask_suspicious].index
        display(df_typed.loc[idx_susp, show_cols].head(10))

else:
    print("[5.9.4] Rating or Genres not found, skipping.")



=== 5.9.4 Rating vs Genres ===
Total rows                                : 24905
Rows with light rating                    : 11710
Rows where Genres contains Hentai/Ecchi   : 2281
Suspicious rows (light rating & H/E)      : 17

Examples - light rating but Hentai/Ecchi in Genres:


,anime_id,Name,Rating,Genres
871,966,Crayon Shin-chan,G - All Ages,"Comedy, Ecchi"
1989,2172,Crayon Shin-chan Movie 15: Arashi wo Yobu Utau...,G - All Ages,"Comedy, Ecchi"
5330,8359,Crayon Shin-chan Movie 04: Henderland no Daibo...,G - All Ages,"Adventure, Ecchi"
5331,8360,Crayon Shin-chan Movie 06: Dengeki! Buta no Hi...,G - All Ages,"Comedy, Slice of Life, Ecchi"
5332,8361,Crayon Shin-chan Movie 07: Bakuhatsu! Onsen Wa...,G - All Ages,"Comedy, Slice of Life, Ecchi"
5336,8365,Crayon Shin-chan Movie 13: Densetsu wo Yobu Bu...,G - All Ages,"Comedy, Slice of Life, Ecchi"
5337,8366,Crayon Shin-chan Movie 14: Densetsu wo Yobu Od...,G - All Ages,"Comedy, Slice of Life, Ecchi"
5338,8367,Crayon Shin-chan Movie 16: Chou Arashi wo Yobu...,G - All Ages,"Comedy, Slice of Life, Ecchi"
8755,22827,Magical★Taruruuto-kun Movie,G - All Ages,"Adventure, Comedy, Fantasy, Ecchi"
8778,22979,Magical★Taruruuto-kun: Sukisuki Takoyaki!,G - All Ages,"Adventure, Comedy, Fantasy, Ecchi"


| ⚠️ EDA Finding                 | Details                                                                                   |
|----------------------------------------|------------------------------------------------------------------------------------------------|
| **Rating vs Genres Conflict**          | Several anime are rated **`G - All Ages`** but contain the **`Ecchi`** genre                  |
| **Severity**                           | **Critical** – violates age-rating classification rules                                       |
| **Suspected Cause**                    | Data entry error in either the `Rating` or `Genres` column                                    |
| **Recommended Action**                 | Immediately review and correct all affected records                                            |